In [1]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

In [2]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

In [1]:
import minsearch

In [2]:
import json
with open('documents.json', 'rt') as f_in:
    docs_raw=json.load(f_in)

In [3]:
docs_raw[1].keys()
docs_raw[1]['documents'][1].keys()

dict_keys(['text', 'section', 'question'])

In [4]:
#docs_raw is a list of 3 dictionaries
print(len(docs_raw))
# each dictionary has two keys
print(docs_raw[0].keys())
# course key has one item
print(docs_raw[0]['course'])
# documents key has a list dictionaries which add up to 435
print(len(docs_raw[0]['documents']))
# each of those dictionaries has three keys namely
print(docs_raw[0]['documents'][0].keys())

3
dict_keys(['course', 'documents'])
data-engineering-zoomcamp
435
dict_keys(['text', 'section', 'question'])


In [5]:
len(docs_raw)
#docs_raw[0]
#len(docs_row[0])

3

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
(documents[0].keys())

dict_keys(['text', 'section', 'question', 'course'])

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]  
)


In [9]:
q = 'the course has already started, can I still enroll?'

In [10]:
index.fit(documents)

In [12]:
boost ={'question':3.0, 'section':0.5}
results= index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5

    
)

In [13]:
#print(results)
results


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [14]:
from openai import OpenAI
client = OpenAI()
#client = OpenAI()

In [15]:
import os
os.environ
envir=os.environ
#envir


In [16]:
if 'OPENAI_API_KEY' in envir:
    print(envir['OPENAI_API_KEY'])
else:
    print(False)

sk-proj-MQGSHWV1ANX6tWDNAt1yT3BlbkFJ5j4UWsrZfBp70gLkyczy


In [17]:
response = client.chat.completions.create(
    #model='gpt-3.5-turbo',
    model='gpt-4',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"As an AI, I don't have information about a specific course. Generally, it depends on the course and institution's policies. Some may allow late enrollment while others might not. It's best to contact the course or program administrator to get accurate information."

In [18]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results
query=search(q)
query

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [19]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [21]:
prompt=build_prompt(query=q, search_results=results)
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related 

In [22]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [23]:
llm(prompt)

'Yes, you can still enroll in the course even if it has already started. You will still be eligible to submit the homeworks, but keep in mind that there will be deadlines for turning in the final projects, so it’s important to manage your time and not leave everything for the last minute.'

In [24]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [25]:
rag(query)

'To run Kafka in the context of producing and consuming messages using provided Java or Python examples, follow these steps:\n\nFor Java Kafka:\n\n1. Navigate to your project directory.\n2. Execute the producer/consumer or any Kafka streams application by running the following command in your terminal:\n   ```sh\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\nFor Python Kafka:\n\n1. Create a virtual environment and install the required packages:\n   ```sh\n   python -m venv env\n   source env/bin/activate  # For Windows, use env\\Scripts\\activate\n   pip install -r ../requirements.txt\n   ```\n2. Ensure Docker images are up and running if needed.\n3. To address specific error like permission denied for build.sh:\n   ```sh\n   chmod +x build.sh  # Run this command in the directory containing build.sh\n   ```\n4. If facing a `ModuleNotFoundError`, use an alternative Kafka Python package:\n   ```sh\n   pip install kafka-pytho

In [26]:
rag('the course has started. Can I still enroll?')

'Yes, you can still enroll in the course even if it has already started. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects.'

In [31]:
rag('Tell me about Kilimanjaro Mountain in Kenya')

"Based on the provided FAQ database, there's no information about Kilimanjaro Mountain within the context given. However, I can provide general information based on common knowledge. Mount Kilimanjaro, the highest peak in Africa, is actually located in Tanzania, not Kenya. It stands at 5,895 meters (19,341 feet) above sea level and is renowned for its snow-capped summit despite being near the equator. The mountain is a popular destination for climbers and offers various routes to its peak, including the Marangu, Machame, and Lemosho routes. It's part of Kilimanjaro National Park, a protected area known for its diverse ecosystems ranging from rainforests to alpine deserts."